In [8]:
from transformers import pipeline
summarizer = pipeline("summarization")
classifier = pipeline("text-classification")
ner_tagger = pipeline("ner", aggregation_strategy="simple")
translator = pipeline("translation_en_to_sv", model="Helsinki-NLP/opus-mt-en-sv") #This one you can change to other language then swedish if you like!
generator = pipeline("text-generation")

print(translator("I love programming in Python"))
print(generator("I love programming in Python"))
print(classifier("I love programming in Python"))
print(ner_tagger("I love programming in Python"))
print(summarizer("I love programming in Python"))

swedish_game_review = """Att följa upp en succé är inte alltid det enklaste, men i fallet God of War: Ragnarök känns det som om Santa Monica Studio trivs som allra bäst när förväntningarna är som störst. Vi har betygsatt Kratos senaste kapitel, och det här är sannerligen en saga du inte vill missa."""

english_game_review = """Santa Monica Studios' follow-up to the 2018 soft reboot is almost here, but does it stack up to its excellent predecessor? Simply put... yes."""

swedish_sports_news = """Det är inte bara på fotbollsplanen som livet leker för Jennifer Falk just nu. Efter succén i Häcken och stora möjligheter att ta över som förstamålvakt i landslaget efter Hedvig Lindahl har hon nu fått livets största gåva."""

celebrity_news = """Television reality star Kim Kardashian West said Friday that she and rapper Kanye West have welcomed their fourth child, a boy born via surrogate.

"He's here and he's perfect!" the 38-year-old Kardashian, who also has her own make-up and fragrance lines, tweeted.

She had announced the couple were expecting in January, telling the US program "Watch What Happens Live With Andy Cohen" that the youngest Kardashian to keep up with was due "sometime soon."

The new baby joins three other siblings: sister Chicago, who was born last year via surrogate, five-year-old sister North, and three-year-old brother Saint."""

print("Swedish Game Review:", swedish_game_review)
print("\nEnglish Game Review:", english_game_review)
print("\nSwedish Sports News:", swedish_sports_news)
print("\nCelebrity News:", celebrity_news)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.poole

[{'translation_text': 'Jag älskar programmering i Python'}]
[{'generated_text': "I love programming in Python and when it comes down for you it means nothing.\n\nHow to Use:\n\nCopy the repo to your existing site and just paste the URL.\n\nYou'll get a list of all required objects, just"}]
[{'label': 'POSITIVE', 'score': 0.9987718462944031}]


Your max_length is set to 142, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


[{'entity_group': 'MISC', 'score': 0.96414167, 'word': 'Python', 'start': 22, 'end': 28}]
[{'summary_text': " I love programming in Python . Python is the language of programming that allows you to write Python code . Python has been used in more than 1,000 languages worldwide . It's a great way to learn Python programming, but it's not the best way to do it ."}]
